In [170]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics.pairwise import cosine_similarity

In [171]:
# Load datasets
# https://www.kaggle.com/datasets/boltcutters/food-allergens-and-allergies?resource=download
allergies = pd.read_csv('FoodData.csv', encoding='latin1')
allergies.head()

,Class,Type,Group,Food,Allergy
0,Plant origin,Nut and seed,Oil seed,Almond,Nut Allergy
1,Plant origin,Fruit,Pome fruit,Apple,Oral Allergy Syndrome
2,Plant origin,Fruit,Stone fruit,Apricot,Stone Fruit Allergy
3,Plant origin,Vegetable,Composite vegetable,Artichoke,Insulin Allergy
4,Plant origin,Vegetable,Liliaceous vegetable,Asparagus,Allium Allergy


In [172]:
# Load datasets
# https://www.kaggle.com/datasets/anoopjohny/new-york-restaurant-menus-and-details
restaurantDishes = pd.read_csv('menu_data.csv', encoding='latin1')
restaurantDishes.head()

,Restaurant,Item,Price
0,Malecon,Potato Salad,$6.00
1,Malecon,Avocado Salad,$6.00
2,Malecon,Mixed Green Salad,$6.00
3,Malecon,Mixed Green Salad w/ Avocado,$7.00
4,Malecon,Seasard Chicken Salad,$17.00


In [173]:
# norm text
allergies = allergies.dropna(subset=['Food', 'Allergy'])
allergies['Food'] = allergies['Food'].str.lower()
allergies['Allergy'] = allergies['Allergy'].str.lower()

# group together foods by allergy category
allergy_map = allergies.groupby("Allergy")["Food"].apply(list).to_dict()
allergy_map


{'allium allergy': ['asparagus',
  'garlic',
  'leek',
  'nira',
  'onion',
  'shallot',
  'welsh'],
 'alpha-gal syndrome': ['cattle', 'deer', 'goat', 'horse', 'pig', 'sheep'],
 'aquagenic urticaria': ['mineral water'],
 'banana allergy': ['banana'],
 'beer allergy': ['hop'],
 'broccoli allergy': ['broccoli'],
 'citrus allergy': ['grapefruit', 'lemon', 'lime', 'orange', 'orange pulp'],
 'corn allergy': ['corn', 'popcorn', 'sweet corn'],
 'cruciferous allergy': ['brussels sprouts',
  'cabbage',
  'cauliflower',
  'chinese cabbage',
  'horseradish',
  'kale',
  'kyona',
  'mustard spinach',
  'qing-geng-cai'],
 'fish allergy': ['eel',
  'globfish',
  'horse mackerel',
  'mackerel',
  'percifomes',
  'salmon',
  'sea bass',
  'sea bream',
  'shelled mollusc',
  'tetraodontiformes',
  'trout',
  'tuna'],
 'gluten allergy': ['barley', 'buckwheat', 'rye', 'wheat'],
 'histamine allergy': ['bamboo shoot', 'bonito', 'ginger', 'okra', 'spinach'],
 'honey allergy': ['honey', 'royal jelly'],
 'hyp

In [174]:
X_text = allergies["Food"].values
y_labels = allergies["Allergy"].values

print("Examples of words in X_text:", X_text[:5])
print("Examples of words in y_labels:", y_labels[:5])

Examples of words in X_text: ['almond' 'apple' 'apricot' 'artichoke' 'asparagus']
Examples of words in y_labels: ['nut allergy' 'oral allergy syndrome' 'stone fruit allergy'
 'insulin allergy' 'allium allergy']


In [175]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X_text)

In [176]:
mlp = MLPClassifier(hidden_layer_sizes=(128, 64),
                    activation='relu',
                    max_iter=800,
                    random_state=42)

mlp.fit(X, y_labels)

MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=800, random_state=42)

In [177]:
restaurantDishes['Item_lower'] = restaurantDishes['Item'].fillna("unknown").astype(str).str.lower()

# vectorize menu items
X_menu = vectorizer.transform(restaurantDishes['Item_lower'])

probas = mlp.predict_proba(X_menu)

allergy_classes = mlp.classes_
proba_df = pd.DataFrame(probas, columns=[f"prob_{a}" for a in allergy_classes])
restaurantDishes = pd.concat([restaurantDishes.reset_index(drop=True), proba_df], axis=1)

restaurantDishes.head()

,Restaurant,Item,Price,Item_lower,prob_allium allergy,prob_alpha-gal syndrome,prob_aquagenic urticaria,prob_banana allergy,prob_beer allergy,prob_broccoli allergy,...,prob_ragweed allergy,prob_rice allergy,prob_salicylate allergy,prob_seed allergy,prob_shellfish allergy,prob_soy allergy,prob_stone fruit allergy,prob_sugar allergy / intolerance,prob_tannin allergy,prob_thyroid
0,Malecon,Potato Salad,$6.00,potato salad,0.000017,0.000437,0.000006,0.000033,0.000032,0.000012,...,0.000054,5.194296e-05,0.000009,0.000076,0.000012,0.000001,0.000504,0.000066,0.000030,0.000069
1,Malecon,Avocado Salad,$6.00,avocado salad,0.000086,0.000043,0.000011,0.000366,0.000004,0.000369,...,0.000084,3.521664e-07,0.000074,0.000064,0.000002,0.000011,0.000092,0.000074,0.000207,0.000005
2,Malecon,Mixed Green Salad,$6.00,mixed green salad,0.000064,0.000066,0.000113,0.000088,0.000033,0.000030,...,0.000039,2.398222e-06,0.000016,0.000146,0.000047,0.000100,0.000013,0.000019,0.000010,0.000041
3,Malecon,Mixed Green Salad w/ Avocado,$7.00,mixed green salad w/ avocado,0.000458,0.000313,0.000188,0.001345,0.000049,0.000636,...,0.000318,1.694340e-06,0.000143,0.000758,0.000027,0.000150,0.000100,0.000168,0.000193,0.000059
4,Malecon,Seasard Chicken Salad,$17.00,seasard chicken salad,0.000022,0.000112,0.000045,0.000078,0.000223,0.000034,...,0.000057,5.121787e-05,0.000270,0.000327,0.000036,0.000131,0.000097,0.000132,0.000109,0.000116


In [178]:
# safety score = lower allergy probability (i.e. safer rest)
restaurantDishes["dish_safety"] = 1 - proba_df.max(axis=1)

# aggregate per restaurant
restaurantSafety = restaurantDishes.groupby("Restaurant")["dish_safety"].mean().reset_index()
restaurantSafety.columns = ["Restaurant", "SafetyScore"]

# rank highest (safest) first
restaurantSafety.sort_values(by="SafetyScore", ascending=False, inplace=True)
restaurantSafety


,Restaurant,SafetyScore
485,il Buco,0.910554
386,San Pietro,0.910554
86,Celeste,0.910554
424,Taqueria San Pedro,0.905824
245,La Nueva Espana,0.898498
...,...,...
289,Modern Gourmet,0.334527
470,Vivi Bubble Tea,0.328997
269,Mango Mango Dessert,0.294569
39,Bagel Works,0.294250


In [179]:
def rank_by_specific_allergy(allergy):
    allergy = allergy.lower()
    col = f"prob_{allergy}"
    
    if col not in restaurantDishes.columns:
        raise ValueError(f"Allergy '{allergy}' not found.")

    restaurantDishes[f"safety_{allergy}"] = 1 - restaurantDishes[col]
    
    ranking = (restaurantDishes
               .groupby("Restaurant")[f"safety_{allergy}"]
               .mean()
               .reset_index()
               .sort_values(by=f"safety_{allergy}", ascending=False))
    
    print(f"\nTop 5 safest restaurants for allergy '{allergy}':")
    print(ranking.head(5).to_string(index=False))
    
    print(f"\nBottom 5 least safe restaurants for allergy '{allergy}':")
    print(ranking.tail(5).to_string(index=False))
    
    return ranking

# plug in diff allergies here
rank_by_specific_allergy("tannin allergy")


Top 5 safest restaurants for allergy 'tannin allergy':
    Restaurant  safety_tannin allergy
Rice To Riches               0.994496
 Pret a Manger               0.989917
       Cottage               0.989671
     Red Mango               0.989224
    Golden Wok               0.989181

Bottom 5 least safe restaurants for allergy 'tannin allergy':
                    Restaurant  safety_tannin allergy
                 Cafe Himalaya               0.906816
               Vivi Bubble Tea               0.906728
             Cupping Room Cafe               0.893426
               Alice's Tea Cup               0.884697
Grace Street Coffee & Desserts               0.848913


,Restaurant,safety_tannin allergy
370,Rice To Riches,0.994496
361,Pret a Manger,0.989917
103,Cottage,0.989671
368,Red Mango,0.989224
173,Golden Wok,0.989181
...,...,...
75,Cafe Himalaya,0.906816
470,Vivi Bubble Tea,0.906728
110,Cupping Room Cafe,0.893426
14,Alice's Tea Cup,0.884697
